# The Epione contact tracing protocol

asecuritysite [link](https://asecuritysite.com/zero/tracking01)

* The [Epione protocol](https://arxiv.org/pdf/2004.13293.pdf) is based on a PSI cardinality proof with PIR using an untrusted server. 
* Alice and Bob both have sets and after the protocol learn only the cardinality of their joint set intersection
* Compared to Apple Google tracker, the Epitone protocol reduces the risk of an actor learning who else is infected (because Bob learns only the intersection cardinality)
* Epione addresses the following concerns with existing contract tracing solutions:
     1. Exposure source. If tokens are publically released, an actor can compare the public log with a local log and guess which actors were tested positive.
     2. Infection status by server. The server should not learn who is infected.
     3. Social graph exposure and user tracking. A central actor should not be able to use reported tokens to create social graphs
     4. False user claims. A user should not be able to use diagnosis status of other uses for their own benefit
* Contact tokens are exchanged using BT and stored in a $\verb|sent_token_list|$ and $\verb|received_token_list|$
* $\verb|sent_token_list|$ is shared with Trent (a trusted healthcare provider) if positive diagnosis.
* $\verb|received_token_list|$ is never shared
* Trent shares the $\verb|sent_token_list|$ with an untrusted server.

Techniques used:
1. Everything in the Apple Google COVID tracer
2. Private set intersection cardinality proof

The protocol can use the following PSI cardinality proof (note that this is not the optimal solution presented in the paper, it is just used to highlight the idea):

* Alice and Bob agree on the curve generator $G$
* Bob has values $a_i$ and matches these with the curve with $\mathbf{A} = [a_i G]$
* Bob generates $r \leftarrow \texttt{rand()}$ and computes $\mathbf{A_r} = [r a_i G]$
* Alice gets $\mathbf{A_r}$, shuffles the elements into $\mathbf{A_{rs}}$, generates a $s \leftarrow \texttt{rand()}$ and elementwise multiplies $s \mathbf{A_{rs}}$
* Bob gets $s \mathbf{A_{rs}}$ and computes $r^{-1} \mod n$ where $n$ is the order of the curve
* Bob computes $(r^{-1} \mod n)s \mathbf{A_{rs}} = s \mathbf{A_s} = [s a_i G]$
* Alice computes $B = s b_i G$ and sends this to Bob
* Bob checks $A \cap B$, which will give him the total number of elements that intersect without knowing the items that intersect. Bob shares the intersection output but not $A_s$

In [1]:
from ecpy.curves import Curve
from Crypto.Util import number
from Crypto.Hash import SHA256
import random
import numpy as np
import datetime
import time
from phe import paillier

In [2]:
def curvePick(x):
    """
    ['stark256',
 'frp256v1',
 'secp521r1',
 'secp384r1',
 'secp256k1',
 'secp256r1',
 'secp224k1',
 'secp224r1',
 'secp192k1',
 'secp192r1',
 'secp160k1',
 'secp160r1',
 'secp160r2',
 'Brainpool-p512t1',
 'Brainpool-p512r1',
 'Brainpool-p384t1',
 'Brainpool-p384r1',
 'Brainpool-p320t1',
 'Brainpool-p320r1',
 'Brainpool-p256r1',
 'Brainpool-p256t1',
 'Brainpool-p224r1',
 'Brainpool-p224t1',
 'Brainpool-p192r1',
 'Brainpool-p192t1',
 'Brainpool-p160r1',
 'Brainpool-p160t1',
 'NIST-P256',
 'NIST-P224',
 'NIST-P192',
 'Ed448',
 'Ed25519',
 'Curve448',
 'Curve25519']
    """
    curve = Curve.get_curve(Curve.get_curve_names()[x])
    return curve

def numfy(s):
    number = 0
    for e in [ord(c) for c in s]:
        number = (number * 0x110000) + e
    return number

def denumfy(number):
    l = []
    while(number != 0):
        l.append(chr(number % 0x110000))
        number = number // 0x110000
    return ''.join(reversed(l))

def generate(s, d, j, i=None):
    '''
    Input seed from CRNG, day in datetime.date format, timeslot as range, with ith element.
    If index is missing, then returns random element in range
    Outputs token
    '''
    if not i == None:
        assert max(j) >= i and min(j) <= i, 'index is outside of range'
        j = str(j[i]).encode('utf-8')
    else:
        j = str(random.choice(j)).encode('utf-8')
    s = str(s).encode('utf-8')
    d = str(int(time.mktime(d.timetuple()))).encode('utf-8')
    a = SHA256.new()
    a.update(s+d+j)
    return a.hexdigest()

**Below is a simple demo of the PSI cardinality proof for easier understanding**

In [11]:
curve = curvePick(4) # 4 is secp256k1
G = curve.generator
order = curve.order
a = [1, 44, 33, 10]
b = [60, 54, 19, 10]

assert all(a[i] > 0 for i in range(len(a))), "Must use positive numbers"
assert all(b[i] > 0 for i in range(len(b))), "Must use positive numbers"
 

s = number.getRandomNBitInteger(256)
r = number.getRandomNBitInteger(256)

raAi = [r * a[i] * G for i in range(len(a))]
sraAi = [s * raAi[i] for i in range(len(a))]

inv_r = pow(r, -1, order)

random.shuffle(sraAi)

sAi = [inv_r * sraAi[i] for i in range(len(a))]  
sBi = [s * b[i] * G for i in range(len(b))]
    
len(np.intersect1d([sBi[i].x for i in range(len(a))], [sAi[i].x for i in range(len(a))]))

1

### Actual protocol

In [17]:
# The epione server generates a keypair
epione_public_key, epione_private_key = paillier.generate_paillier_keypair()

## Alice and Bob run into one another
# Alice generates a token
random.seed(123)
seed_alice = random.getrandbits(256)
random.seed()
date_alice = datetime.date(2020, 12, 25)
range_alice = range(144)
proxy_alice = 1
token_alice = generate(seed_alice, date_alice, range_alice, proxy_alice)

# Bob generates a token
random.seed(456)
seed_bob = random.getrandbits(256)
random.seed()
date_bob = datetime.date(2020, 12, 25)
range_bob = range(144)
proxy_bob = 1
token_bob = generate(seed_bob, date_bob, range_bob, proxy_bob)

# Charlie generates a token
random.seed(789)
seed_charlie = random.getrandbits(256)
random.seed()
date_charlie = datetime.date(2020, 12, 25)
range_charlie = range(144)
proxy_charlie = 1
token_charlie = generate(seed_charlie, date_charlie, range_charlie, proxy_charlie)

# Tokens are exchanged
alice_received_list = [token_bob, token_charlie]
bob_received_list = [token_alice, token_charlie]
charlie_received_list = [token_alice, token_bob] # use if Charlie should get tested
# charlie_received_list = generate(number.getRandomNBitInteger(256), datetime.date.today(), range(144), 1) # use if charlie should not get tested

In [18]:
# Alice tests positive and submits her seed with the Epione server public key to Trent the healthcare provider
trent_from_alice = epione_public_key.encrypt(123) # use numfy if seed is string

# Bob tests positive and submits his seed with the Epione server public key to Trent the healthcare provider
trent_from_bob = epione_public_key.encrypt(456)

# Trent sends the list of encrypted seeds to Epione
epione_from_trent = [trent_from_alice, trent_from_bob]

# Epione server now generates all the tokens possible
all_tokens = []
for i in epione_from_trent:
    random.seed(epione_private_key.decrypt(i))
    for x in range(14):
        all_tokens.append(generate(random.getrandbits(256), datetime.date(2020, 12, 25) - datetime.timedelta(days=x), range(144), 1))
    random.seed()

# assert len(np.intersect1d(all_tokens, charlie_received_list)) != 0

In [19]:
# Charlie and Epione agree on a curve and a generator
curve = curvePick(4) # 4 is secp256k1
G = curve.generator
order = curve.order

# Charlie generates random value s and turns his values into numbers
s = number.getRandomNBitInteger(256)
charlie_received_list_numbers = [numfy(charlie_received_list[i]) for i in range(len(charlie_received_list))]

# Epione generates random value r and turns values into numbers
r = number.getRandomNBitInteger(256)
inv_r = pow(r, -1, order)
epione_list_numbers = [numfy(all_tokens[i]) for i in range(len(all_tokens))]
r_epione = [r * epione_list_numbers[i] * G for i in range(len(epione_list_numbers))] #sent to Charlie

# Charlie now generates his set
sr_epione = [s * r_epione[i] for i in range(len(epione_list_numbers))]
random.shuffle(sr_epione) #this is sent to sever
s_charlie = [s * charlie_received_list_numbers[i] * G for i in range(len(charlie_received_list_numbers))] #this is sent to sever

# Epione server now does its part
s_epione = [inv_r * sr_epione[i] for i in range(len(sr_epione))]

if len(np.intersect1d([s_charlie[i].x for i in range(len(charlie_received_list_numbers))], [s_epione[i].x for i in range(len(s_epione))])) != 0:
    print('Charlie should get tested')
else:
    print('Charlie does not need to get tested')

Charlie should get tested
